# **Задание 1**

In [ ]:
import numpy as np

class CNB:

  def __init__(self):
    self.apr = {}
    self.feature_probs = {}

  def fit(self, X_train, y_train):

    classesNumber = len(set(y_train))

    #априорная вероятность класса
    self.all_classes = list(set(y_train))
    self.apr = [[i, list(y_train).count(i)/len(y_train)] for i in self.all_classes]

    self.all_classes = np.array(self.all_classes)

    X_train =  np.array(X_train)
    y_train = np.array(y_train)

    self.apr = dict(self.apr)

    #вероятность признаков каждого класса
    extra_arr = []
    for j in range(len(X_train[0])):
      extra_arr.append([])
      for i in range(len(X_train)):
        extra_arr[j].append(X_train[i][j])
      extra_arr[j] = list(set(extra_arr[j]))


    all = dict(Counter(y_train).most_common())
    for one_class in list(set(y_train)):
      arr = []
      for j in range(len(X_train[0])):
        arr.append({ex : 0.0 for ex in extra_arr[j]})
        for i in range(len(X_train)):
          if y_train[i] == one_class:
            arr[j][X_train[i][j]] += 1
        for i in arr[j]:
          arr[j][i] = arr[j][i]/all[one_class]
      self.feature_probs[one_class] = arr

  def predict_proba(self, X):
    a = 1
    probs = np.zeros((X.shape[0], len(self.all_classes)))

    for i, one_class in enumerate(self.all_classes):
      # Вероятности признаков для текущего класса
      feature_prob = self.feature_probs[one_class]
      probs_class = np.full(X.shape[0], self.apr[one_class])
      for j in range(X.shape[1]):
          # Вычисляю вероятности для каждого признака в данном классе
          probs_class *= [feature_prob[j].get(value, a) for value in X[:, j]]
      probs[:, i] = probs_class
    # Возвращаю нормализованные вероятности для каждого класса, поделив на сумму вероятностей по строкам
    return probs/np.sum(probs, axis=1, keepdims=True)

  def predict(self, X):
    return np.argmax(self.predict_proba(X), axis=1)

  def score(self, X, y):
    return np.mean(self.predict(X) == y)


In [ ]:
from sklearn.utils import Bunch
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import CategoricalNB

myDataset = pd.read_csv('/content/drive/MyDrive/Machine learning/Юрченко.csv')

myDataset_numpy = myDataset.to_numpy()


myDataset.drop(columns = ["Unnamed: 0"])

dataset = Bunch()
encoder = OrdinalEncoder()
data_encoded = encoder.fit_transform(myDataset_numpy)

dataset.data = data_encoded[:, :-1]
dataset.target = data_encoded[:, -1]

dataset.DESCR = "MyDataset"
dataset.target_names = np.unique(dataset.target)


X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.3)


test = CategoricalNB()
test.fit(X_train, y_train)
accuracy = accuracy_score(y_test, test.predict(X_test)) * 100
print(accuracy)

79.33333333333333


# Задание 2

In [ ]:
from sklearn.datasets import load_digits
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

digits = load_digits()
canser = load_breast_cancer()
iris = load_iris()
wine = load_wine()

A, B, C, D = GaussianNB(), MultinomialNB(), ComplementNB(), BernoulliNB()
everything = {"Цифры": digits, "Рак":canser, "Ирисы":iris, "Вино":wine, "Мой датасет": dataset}
methods = {"GaussianNB":A, "MultinomialNB": B, "ComplementNB" :C, "BernoulliNB": D}


for one in everything:
  print("--------------------------------------------------------------------------------------")
  print("Датасет ", one)
  best_method = "GaussianNB"
  best_accuracy = 0
  for method in methods:

    X = everything[one].data
    y = everything[one].target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    methods[method].fit(X_train, y_train)
    accuracy = accuracy_score(y_test, methods[method].predict(X_test))
    print("Метод: ", method)
    print("Точность методом accuracy_score:", accuracy)
    if best_accuracy < accuracy:
      best_accuracy = accuracy
      best_method = method
  if one == "Мой датасет":
    test = CategoricalNB()
    test.fit(X_train, y_train)
    accuracy = accuracy_score(y_test, test.predict(X_test))
    print("Метод: ", "CategoricalNB")
    print("Точность методом accuracy_score:", accuracy)
    if best_accuracy < accuracy:
      best_accuracy = accuracy
      best_method = "CategoricalNB"
  print("--------------------------------------------------------------------------------------")
  print(f"Лучший метод {best_method} с точностью {best_accuracy}")
  print("--------------------------------------------------------------------------------------")





--------------------------------------------------------------------------------------
Датасет  Цифры
Метод:  GaussianNB
Точность методом accuracy_score: 0.8351851851851851
Метод:  MultinomialNB
Точность методом accuracy_score: 0.8981481481481481
Метод:  ComplementNB
Точность методом accuracy_score: 0.825925925925926
Метод:  BernoulliNB
Точность методом accuracy_score: 0.8666666666666667
--------------------------------------------------------------------------------------
Лучший метод MultinomialNB с точностью 0.8981481481481481
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
Датасет  Рак
Метод:  GaussianNB
Точность методом accuracy_score: 0.9181286549707602
Метод:  MultinomialNB
Точность методом accuracy_score: 0.8596491228070176
Метод:  ComplementNB
Точность методом accuracy_score: 0.8888888888888888
Метод:  BernoulliNB
Точность методом accuracy_score: 0.6666

# Тест

In [ ]:
import pandas as pd
from sklearn.naive_bayes import CategoricalNB as cnb
from sklearn.model_selection import train_test_split

myDataset = pd.read_csv('/content/drive/MyDrive/Machine learning/Юрченко.csv')

x_1 = list(myDataset['colour'])
x_2 = list(myDataset['height'])
x_3 = list(myDataset['shape'])
x_4 = list(myDataset['density'])
x_5 = list(myDataset['available'])

y_1 = list(myDataset['demand'])

def preproc(arr):
  extra = list(set(arr))
  res = []
  for i in range(len(arr)):
    indx = extra.index(arr[i])
    res.append(indx)
  return res, extra

def unpreproc(arr, key):
  res = []
  for i in range(len(arr)):
    res.append(key[arr[i]])
  return res

_x_1, key1 = preproc(x_1)
#print("\n", x_1)
#print("\n", key1)
#print("\n", _x_1)
#print("\n", unpreproc(_x_1, key1))
_x_2, key2 = preproc(x_2)
_x_3, key3 = preproc(x_3)
_x_4, key4 = preproc(x_4)
_x_5, key5 = preproc(x_5)

_y_1, y_key1 = preproc(y_1)

X = [ [_x_1[i], _x_2[i], _x_3[i], _x_4[i], _x_5[i]] for i in range(0, len(_x_1))]
y = _y_1

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3)

#print(X)

test = cnb()
test.fit(X_train, y_train)

test.score(X_test,y_test)



0.7733333333333333

In [ ]:
import numpy as np
from collections import Counter


X = [[3, 5, 6], [2, 4, 6], [1, 3, 6], [2, 5, 6]]
y = [1, 4, 4, 2]

prob = {}
for one_class in list(set(y)):
  arr = []
  for j in range(len(X[0])):
    arr.append([])
    for i in range(len(X)):
      if y[i] == one_class:
        arr[j].append(X[i][j])
  for k in range(len(arr)):
    arr[k] = Counter(arr[k]).most_common()
  prob[one_class] = arr

extra_arr = []
for j in range(len(X[0])):
    extra_arr.append([])
    for i in range(len(X)):
      extra_arr[j].append(X[i][j])
    extra_arr[j] = list(set(extra_arr[j]))

all = dict(Counter(y).most_common())
feature_probs = {}


arr = []
for one_class in list(set(y)):
  arr = []
  for i in range(len(prob[one_class])):
    arr.append({})
    for j in range(len(prob[one_class][i])):
      arr[i][prob[one_class][i][j][0]] = prob[one_class][i][j][1]/all[one_class]
  feature_probs[one_class] = arr

# for one_class in list(set(y)):
#   for i in range(len(feature_probs[one_class])):
#     for j in range(len(extra_arr)):
#       for k in range(len(extra_arr[j])):
#         if extra_arr[j][k] not in feature_probs[one_class][i]:
#           feature_probs[one_class][i][extra_arr[j][k]] = 0.0


print(prob)











{1: [[(3, 1)], [(5, 1)], [(6, 1)]], 2: [[(2, 1)], [(5, 1)], [(6, 1)]], 4: [[(2, 1), (1, 1)], [(4, 1), (3, 1)], [(6, 2)]]}


In [ ]:
import numpy as np
from collections import Counter


X = [[3, 5, 6], [2, 4, 6], [2, 3, 6], [2, 5, 6]]
y = [1, 4, 4, 2]

extra_arr = []
for j in range(len(X[0])):
  extra_arr.append([])
  for i in range(len(X)):
    extra_arr[j].append(X[i][j])
  extra_arr[j] = list(set(extra_arr[j]))


all = dict(Counter(y).most_common())
prob = {}
for one_class in list(set(y)):
  arr = []
  for j in range(len(X[0])):
    arr.append({ex : 0.0 for ex in extra_arr[j]})
    for i in range(len(X)):
      if y[i] == one_class:
        arr[j][X[i][j]] += 1
    for i in arr[j]:
      arr[j][i] = arr[j][i]/all[one_class]
  prob[one_class] = arr




print(prob)

{1: [{2: 0.0, 3: 1.0}, {3: 0.0, 4: 0.0, 5: 1.0}, {6: 1.0}], 2: [{2: 1.0, 3: 0.0}, {3: 0.0, 4: 0.0, 5: 1.0}, {6: 1.0}], 4: [{2: 1.0, 3: 0.0}, {3: 0.5, 4: 0.5, 5: 0.0}, {6: 1.0}]}
